In [1]:
import dask.dataframe as dd
from dask.dataframe.utils import make_meta
from neo4j import GraphDatabase
from dask.distributed import Client, LocalCluster, get_worker
import dask
import dill
import os

In [2]:
def addActivity(tx, activityName):
    tx.run("MERGE (p:Activity {name: $activityName})", activityName=activityName)

def setLinks(row):
    row['predecessor'] = row['activityNameEN'].shift(1); #lag(1)
    row['successor'] = row['activityNameEN'].shift(-1); #lead(1)
    return row;

def getNeo4jCredentials():
    return {
        "host": "bolt://34.239.227.164:7687",
        "user": "neo4j",
        "password": "members-goals-technique"
    }

In [3]:
columnTypes = {
    'case:IDofConceptCase': 'string',
    'case:Includes_subCases': 'string',
    'case:Responsible_actor': 'string',
    'case:caseProcedure': 'string',
    'dateStop': 'string'
}
df = dd.read_csv('BPIC15_1.csv', dtype=columnTypes)
client = Client(n_workers=1)

2022-12-03 17:04:31,737 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-waa2f_51', purging


In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 5,Total memory: 7.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38429,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 5
Started: Just now,Total memory: 7.67 GiB
Comm: tcp://127.0.0.1:38731,Total threads: 5
Dashboard: http://127.0.0.1:39855/status,Memory: 7.67 GiB
Nanny: tcp://127.0.0.1:43541,


In [5]:
creds = getNeo4jCredentials()
driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))

In [6]:
dill_file = os.path.join(os.getcwd(), 'dbconn.dill')
with open(dill_file, 'wb') as f:
    dill.dump(driver, f)

In [7]:
scattered_conn = client.scatter(dill.dumps(driver), broadcast=True)

In [8]:
scattered_file = client.scatter(dill_file, broadcast=True)

In [9]:
# with open(dill_file, 'rb') as f:
#     result = dill.load(f)

In [10]:
# def connect_worker_db():
#     connection = dill.load(open(os.path.join(os.getcwd(), 'dbconn.dill'), 'rb'))
#     worker = get_worker()
#     worker.driver = connection

In [11]:
# client.register_worker_callbacks(connect_worker_db)

In [12]:
df['successor'] = ''
df['predecessor'] = ''
df = df.sort_values(by='time:timestamp').groupby('case:concept:name').apply(setLinks, meta=df)

In [13]:
result = df['activityNameEN'].unique()

In [19]:
def experiment(file):
    driver = dill.loads(file)
#     with driver.session() as session:
#         result.apply(lambda activityName: addActivity(session, activityName), meta=('activityNameEN', 'object')).compute()
#     session.close()
    driver.close()
    return dill.__version__

In [20]:
result = client.submit(experiment, scattered_conn)

In [21]:
result

<Future: pending, key: experiment-15a37d46dbb26c8a542575040df5e4ad>

In [ ]:
client.gather(result)